In [1]:
import boto3
import os
from pathlib import Path
from tqdm import tqdm

In [2]:
def download_s3_with_structure(bucket_name, s3_folder, local_dir):
    """
    S3 버킷의 특정 폴더에서 원본 폴더 구조를 유지하며 모든 파일을 다운로드합니다.
    
    Args:
        bucket_name (str): S3 버킷 이름
        s3_folder (str): S3 내의 폴더 경로
        local_dir (str): 로컬에 저장할 디렉토리 경로
    """

In [2]:
    s3 = boto3.client('s3')
    
    # S3 폴더 구조를 그대로 로컬에 생성
    local_base = os.path.join(local_dir, Path(s3_folder).parts[-3], Path(s3_folder).parts[-2], Path(s3_folder).parts[-1])
    os.makedirs(local_base, exist_ok=True)
    
    # S3 객체 리스트 가져오기
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=s3_folder)
    
    # 전체 파일 개수 계산
    total_files = sum(1 for page in pages for _ in page.get('Contents', []))
    
    # 다시 페이지네이터 생성
    pages = paginator.paginate(Bucket=bucket_name, Prefix=s3_folder)
    
    print(f"총 {total_files}개의 파일을 다운로드합니다.")
    print(f"다운로드 경로: {local_base}")
    
    # tqdm으로 진행률 표시하며 다운로드
    with tqdm(total=total_files, desc="Downloading") as pbar:
        for page in pages:
            if 'Contents' not in page:
                continue
                
            for obj in page['Contents']:
                key = obj['Key']
                if key.endswith('/'):  # 폴더인 경우 스킵
                    continue
                    
                # 원본 경로 구조 유지
                local_path = os.path.join(local_dir, *Path(key).parts[:-1])
                local_file_path = os.path.join(local_dir, *Path(key).parts)
                
                # 필요한 디렉토리 생성
                os.makedirs(local_path, exist_ok=True)
                
                # 파일 다운로드
                s3.download_file(bucket_name, key, local_file_path)
                pbar.update(1)
                
    print(f"\n다운로드 완료!")
    return local_base

In [3]:
# 설정
BUCKET_NAME = 'smwu-cv-data'
S3_FOLDER = 'data/다이캐스팅/g2/'
LOCAL_DIR = 'Downloaded_data'  # 최상위 폴더명

# 다운로드 실행
download_path = download_s3_with_structure(BUCKET_NAME, S3_FOLDER, LOCAL_DIR)

# 다운로드된 파일 확인
print("\n다운로드된 파일 구조:")
for path in Path(download_path).rglob('*'):
    depth = len(path.relative_to(download_path).parts)
    prefix = '  ' * depth
    print(f"{prefix}- {path.name}")

총 1917개의 파일을 다운로드합니다.
다운로드 경로: Downloaded_data/data/다이캐스팅/g2


Downloading: 100%|██████████| 1917/1917 [06:54<00:00,  4.63it/s]


다운로드 완료!

다운로드된 파일 구조:
  - dataset_2023-10-17
  - dataset_2024-01-30
  - dataset_2024-01-15
    - OK
    - NG
      - 5
      - 2
      - 1
      - 4
      - 3
        - 20231013-202448882.png
        - 20231013-205055244.png
        - 20231013-210304433.png
        - 20231013-203333690.png
        - 20231013-205504653.png
        - 20231013-205637224.png
        - 20231013-205420634.png
        - 20231013-205941058.png
        - 20231013-205831856.png
        - 20231013-203122840.png
        - 20231013-205726885.png
        - 20231013-203039324.png
        - 20231013-205442694.png
        - 20231013-204843287.png
        - 20231013-203417306.png
        - 20231013-204035650.png
        - 20231013-202211642.png
        - 20231013-205117203.png
        - 20231013-210238242.png
        - 20231013-210315715.png
        - 20231013-204821227.png
        - 20231013-210004830.png
        - 20231013-204357212.png
        - 20231013-204759268.png
        - 20231013-205252696.png
        - 2023

In [ ]:
#데이터셋 폴더 구조 만들기


import os
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def restructure_dataset(source_path, destination_path, test_size=0.2, val_size=0.1):
    """
    데이터셋을 훈련/검증/테스트 세트로 나누고 재구성합니다.
    
    Args:
        source_path (str): 원본 데이터셋 경로
        destination_path (str): 재구성될 데이터셋 저장 경로
        test_size (float): 테스트 세트 비율
        val_size (float): 검증 세트 비율
    """
    # 목적지 디렉토리 생성
    for split in ['train', 'val', 'test']:
        for label in ['OK', 'NG']:
            os.makedirs(os.path.join(destination_path, split, label), exist_ok=True)
    
    # 모든 이미지 파일의 경로를 수집
    ok_images = []
    ng_images = []
    
    # 데이터셋 디렉토리들을 순회
    dataset_dirs = [d for d in Path(source_path).iterdir() if d.is_dir() and d.name.startswith('dataset_')]
    
    for dataset_dir in dataset_dirs:
        # OK 이미지 수집
        ok_dir = dataset_dir / 'OK'
        if ok_dir.exists():
            for angle_dir in ok_dir.iterdir():
                if angle_dir.is_dir():
                    ok_images.extend(list(angle_dir.glob('*.*')))
        
        # NG 이미지 수집
        ng_dir = dataset_dir / 'NG'
        if ng_dir.exists():
            for angle_dir in ng_dir.iterdir():
                if angle_dir.is_dir():
                    ng_images.extend(list(angle_dir.glob('*.*')))
    
    # 데이터 분할
    # 먼저 train+val과 test로 분할
    ok_trainval, ok_test = train_test_split(ok_images, test_size=test_size, random_state=42)
    ng_trainval, ng_test = train_test_split(ng_images, test_size=test_size, random_state=42)
    
    # train+val을 다시 train과 val로 분할
    val_ratio = val_size / (1 - test_size)  # 남은 데이터 중에서의 검증 세트 비율
    ok_train, ok_val = train_test_split(ok_trainval, test_size=val_ratio, random_state=42)
    ng_train, ng_val = train_test_split(ng_trainval, test_size=val_ratio, random_state=42)
    
    # 데이터 복사 함수
    def copy_files(file_list, label, split):
        for file in tqdm(file_list, desc=f'Copying {split}/{label}'):
            destination = os.path.join(destination_path, split, label, file.name)
            shutil.copy2(file, destination)
    
    # 파일 복사 실행
    print("데이터셋 재구성 중...")
    copy_files(ok_train, 'OK', 'train')
    copy_files(ok_val, 'OK', 'val')
    copy_files(ok_test, 'OK', 'test')
    copy_files(ng_train, 'NG', 'train')
    copy_files(ng_val, 'NG', 'val')
    copy_files(ng_test, 'NG', 'test')
    
    # 데이터셋 통계 출력
    print("\n데이터셋 구성 완료!")
    print(f"훈련 세트: OK {len(ok_train)}장, NG {len(ng_train)}장")
    print(f"검증 세트: OK {len(ok_val)}장, NG {len(ng_val)}장")
    print(f"테스트 세트: OK {len(ok_test)}장, NG {len(ng_test)}장")

# 사용 예시
SOURCE_PATH = "Downloaded_data/data/다이캐스팅/g2"  # 원본 데이터셋 경로
DEST_PATH = "Original_data_for_mod"   # 재구성될 데이터셋 경로

restructure_dataset(SOURCE_PATH, DEST_PATH)